In [29]:
import numpy as np
import gym
import keras
from sklearn.neural_network import MLPRegressor
import sklearn
import warnings
warnings.filterwarnings("ignore")

In [51]:
class DeepNeuroevolution():
    
    def __init__(self, env, n_individuals, n_actions, n_parents, n_features, nn_architecture, reward_goal):
        self._env = env
        self._n_parents = n_parents
        self._n_actions = n_actions
        self._n_features = n_features
        self._n_individuals = n_individuals
        self._nn_architecture = nn_architecture
        self._reward_goal = reward_goal
        self._best_score = -10**10
    
    def find_optimal_network(self):
        self._create_first_generation()
        while not self._is_finished():
            self._create_new_generation()
    
    def _is_finished(self):
        return self._best_score > self._reward_goal
    
    def _evaluate_network(self, mlp, iterations):
        score = 0
        env = self._env
        for _ in range(iterations):
            state = env.reset()
            done = False
            while not done:
                action = np.argmax(mlp.predict([state]))
                state, reward, done, info = env.step(action)
                score += reward
        return score / iterations
    
    def _evaluate_generation(self):
        for idx, mlp in enumerate(self._current_population):
            score = self._evaluate_network(mlp[0], 10)
            self._current_population[idx][1] = score
    
    def _create_first_generation(self):
        self._current_population = []
        for _ in range(self._n_individuals):
            mlp = MLPRegressor(hidden_layer_sizes = (10,), alpha=10**-10, max_iter=1)
            mlp.fit([np.random.randn(self._n_features)], [np.random.randn(self._n_actions)])
            self._current_population.append([mlp,0])
    
    def _create_parents(self):
        parents = sorted(self._current_population, key=lambda x: x[1])[:self._n_parents]
        for idx, mlp in enumerate(parents):
            score = self._evaluate_network(mlp[0], 100)
            parents[idx][1] = score
        parents.sort(key=lambda x:x[1])
        return parents
    
    def _create_new_nn(self, weights, biases):
        mlp = MLPRegressor(hidden_layer_sizes = (10,), alpha=10**-10, max_iter=1)
        mlp.fit([np.random.randn(self._n_features)], [np.random.randn(self._n_actions)])
        mlp.coefs_ = weights
        mlp.intercepts_ = biases
        return mlp
    
    def _compute_new_weights(self, parent):
        weights = parent.coefs_
        biases = parent.intercepts_
        new_weights = []
        new_biases = []
        for weight in weights:
            shape = weight.shape
            new_weights.append(weight + np.random.randn(shape[0], shape[1]))
        for bias in biases:
            new_biases.append(bias + np.random.randn(bias.shape[0]))
        return new_weights, new_biases
    
    def _create_new_population(self, parents):
        new_generation = [parents[0]]
        for _ in range(self._n_individuals-1):
            idx = np.random.randint(len(parents))
            weights, biases = self._compute_new_weights(parents[idx][0])
            mlp = self._create_new_nn(weights, biases)
            new_generation.append([mlp, 0])
        self._current_population = new_generation
        
    def _create_new_generation(self):
        self._evaluate_generation()
        parents = self._create_parents()
        print(parents)
        self._best_score = max(self._best_score, parents[0][1])
        print(parents)
        self._create_new_population(parents)

In [52]:
env = gym.make('CartPole-v0')
dne = DeepNeuroevolution(env, 30, 2,10, 4, (10,), 200)
dne.find_optimal_network()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[[MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.19], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_sta

[[MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.27], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.28], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
    

[[MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.26], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.28], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
    

[[MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.26], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.3], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
     

[[MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.27], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.31], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
    

...

[[MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.21], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.28], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
    

[[MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.3], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.3], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
      

[[MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.07], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.29], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
    

[[MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.11], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), 9.3], [MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
     

KeyboardInterrupt: 

In [3]:
mlp = MLPRegressor(hidden_layer_sizes = (10), alpha=10**-10, max_iter=1)

In [4]:
mlp.fit([[1,2]], [[1,2]])
a = mlp.coefs_

/home/janus/repos/rl-zoo/venv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
b = []
for i in range(len(a)):
    b.append(np.zeros(a[i].shape))

In [5]:
a

[array([[-0.60148067, -0.5043865 ,  0.15464397,  0.68612686, -0.64469093,
         -0.34605044,  0.45728095, -0.70045368, -0.11568803,  0.6485092 ],
        [ 0.49510031,  0.49250125,  0.50033735, -0.33294515,  0.04533942,
          0.55863997, -0.43653175,  0.51707867, -0.12598904, -0.52247543]]),
 array([[ 0.00582509,  0.10483315],
        [ 0.34541089,  0.5690742 ],
        [ 0.22727794, -0.02195987],
        [-0.32846436,  0.24189471],
        [-0.04220763,  0.13375209],
        [ 0.34091695,  0.59917176],
        [-0.65725891,  0.45090632],
        [ 0.08897465,  0.20182847],
        [ 0.38220394,  0.05880929],
        [-0.16636981, -0.02431179]])]

In [6]:
mlp2 = sklearn.base.clone(mlp)

In [ ]:
mlp2 = MLPRegressor(hidden_layer_sizes = (10), alpha=10**-10, max_iter=1)
mlp2.coefs_ = b

In [ ]:
mlp2.fit([[1,2]], [[1,2]])

In [8]:
mlp2.coefs_ = mlp.coefs_

In [9]:
mlp2.coefs_


[array([[-0.60148067, -0.5043865 ,  0.15464397,  0.68612686, -0.64469093,
         -0.34605044,  0.45728095, -0.70045368, -0.11568803,  0.6485092 ],
        [ 0.49510031,  0.49250125,  0.50033735, -0.33294515,  0.04533942,
          0.55863997, -0.43653175,  0.51707867, -0.12598904, -0.52247543]]),
 array([[ 0.00582509,  0.10483315],
        [ 0.34541089,  0.5690742 ],
        [ 0.22727794, -0.02195987],
        [-0.32846436,  0.24189471],
        [-0.04220763,  0.13375209],
        [ 0.34091695,  0.59917176],
        [-0.65725891,  0.45090632],
        [ 0.08897465,  0.20182847],
        [ 0.38220394,  0.05880929],
        [-0.16636981, -0.02431179]])]

In [10]:
mlp2.fit([[1,2]], [[1,2]])

/home/janus/repos/rl-zoo/venv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=10, learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [12]:
mlp2.coefs_ = mlp.coefs_

In [15]:
mlp2.predict([[1,2]])

array([[0.28722212, 1.36535303]])